In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [ ]:
def get_accuracy(y_test: pd.DataFrame, y_pred: np.array):
    y_test = y_test.values.flatten()
    
    class_labels = list(set(y_test))
    class_labels.sort()  
    accuracy = []

    for label in class_labels:
        true_positives = sum(1 for yt, yp in zip(y_test, y_pred) if yt == label and yp == label)
        total_samples = sum(1 for yt in y_test if yt == label)
        
        accuracy_temp = true_positives / total_samples if total_samples > 0 else 0.0
        accuracy.append(accuracy_temp)
    
    return accuracy

In [ ]:
sheet_name = ['IPB','IAB','OIB','MORB','BABB','OFB','CFB']

# columns = ['ID','TIO2(WT%)','P2O5(WT%)','NB(PPM)','TA(PPM)','ZR(PPM)','HF(PPM)','Y(PPM)','LA(PPM)','CE(PPM)','PR(PPM)','ND(PPM)','SM(PPM)','EU(PPM)','GD(PPM)','HO(PPM)','ER(PPM)','YB(PPM)','LU(PPM)','DY(PPM)','TB(PPM)','CR(PPM)','NI(PPM)']
columns = ['ID','SIO2(WT%)','TIO2(WT%)','AL2O3(WT%)','FEOT(WT%)','CAO(WT%)','MGO(WT%)','MNO(WT%)','K2O(WT%)','NA2O(WT%)','P2O5(WT%)','RB(PPM)','SR(PPM)','BA(PPM)','TH(PPM)','U(PPM)','NB(PPM)','TA(PPM)','ZR(PPM)','HF(PPM)','Y(PPM)','LA(PPM)','CE(PPM)','PR(PPM)','ND(PPM)','SM(PPM)','EU(PPM)','GD(PPM)','HO(PPM)','ER(PPM)','YB(PPM)','LU(PPM)','DY(PPM)','TB(PPM)','CR(PPM)','NI(PPM)']

data_train_path = 'data_train.xlsx'
data_test_path = 'data_test.xlsx'

data_train = []
data_test = []

for i in range(len(sheet_name)):
    data_train_temp = pd.read_excel(data_train_path, sheet_name=sheet_name[i])
    data_train_temp = data_train_temp[columns]
    data_train_temp['ID_new'] = i
    data_train.append(data_train_temp)
               
    data_test_temp = pd.read_excel(data_test_path, sheet_name=sheet_name[i])
    data_test_temp = data_test_temp[columns]
    data_test_temp['ID_new'] = i
    data_test.append(data_test_temp)

In [ ]:
data = data_train
train = pd.concat([data[0], data[1], data[2], data[3], data[4], data[5], data[6]], axis=0)
train = train.sample(frac=1, random_state=10)

data = data_test
test = pd.concat([data[0], data[1], data[2], data[3], data[4], data[5], data[6]], axis=0)
test = test.sample(frac=1, random_state=12)

In [ ]:
X_train = train.iloc[:,1:-1]
y_train = train.iloc[:,-1:]

X_test = test.iloc[:,1:-1]
y_test = test.iloc[:,-1:]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

accuracy_dict = {}
accuracy_dict['category'] = ['IPB','IAB','OIB','MORB','BABB','OFB','CFB','Overall']

precision_dict = {}
precision_dict['category'] = ['IPB','IAB','OIB','MORB','BABB','OFB','CFB','Overall']

recall_dict = {}
recall_dict['category'] = ['IPB','IAB','OIB','MORB','BABB','OFB','CFB','Overall']

f1_score_dict = {}
f1_score_dict['category'] = ['IPB','IAB','OIB','MORB','BABB','OFB','CFB','Overall']

In [ ]:
# from sklearn.model_selection import GridSearchCV

# model = svm.SVC()
# param = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': [0.1, 1, 10]}

# model = RandomForestClassifier()
# param = {'n_estimators': [100, 200, 300],'criterion': ['gini','entropy','log_loss'],'max_depth': [10, 20, 30]}

# model = XGBClassifier()
# param = {'booster':['gbtree','dart'], 'n_estimators': [50,100,200,500]}

# grid_search = GridSearchCV(estimator=model, param_grid=param, cv=5, scoring='accuracy')
# grid_search.fit(X_train, y_train)

# best_params = grid_search.best_params_
# best_score = grid_search.best_score_
# print("Best Hyperparameters:", best_params)
# print("Best score:", best_score)

In [ ]:
SVM = svm.SVC(C=10, kernel='rbf')
SVM.fit(X_train, y_train)

y_pred_SVM = SVM.predict(X_test)

accuracy_SVM = accuracy_score(y_test, y_pred_SVM)
precision_SVM = precision_score(y_test, y_pred_SVM, average='macro')
recall_SVM = recall_score(y_test, y_pred_SVM, average='macro')
f1_SVM = f1_score(y_test, y_pred_SVM, average='macro')

print('accuracy_SVM = ',accuracy_SVM)
print('precision_SVM = ',precision_SVM)
print('recall_SVM = ',recall_SVM)
print('f1_SVM = ',f1_SVM)
print()

accuracy = get_accuracy(y_test, y_pred_SVM)
precision = precision_score(y_test, y_pred_SVM, average=None, labels=range(len(train['ID'].unique())))
recall = recall_score(y_test, y_pred_SVM, average=None, labels=range(len(train['ID'].unique())))
f1 = f1_score(y_test, y_pred_SVM, average=None, labels=range(len(train['ID'].unique())))

# print('accuracy:',accuracy)
# print('precision:',list(precision))
# print('recall:',list(recall))
# print('f1_score:',list(f1))

accuracy_dict['SVM'] = accuracy
accuracy_dict['SVM'].append(accuracy_SVM)

precision_dict['SVM'] = list(precision)
precision_dict['SVM'].append(precision_SVM)

recall_dict['SVM'] = list(recall)
recall_dict['SVM'].append(recall_SVM)

f1_score_dict['SVM'] = list(f1)
f1_score_dict['SVM'].append(f1_SVM)

In [ ]:
rf = RandomForestClassifier(n_estimators=300, criterion='entropy', max_depth=30)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='macro')
recall_rf = recall_score(y_test, y_pred_rf, average='macro')
f1_rf = f1_score(y_test, y_pred_rf, average='macro')

print('accuracy_rf = ',accuracy_rf)
print('precision_rf = ',precision_rf)
print('recall_rf = ',recall_rf)
print('f1_rf = ',f1_rf)
print()

accuracy = get_accuracy(y_test, y_pred_rf)
precision = precision_score(y_test, y_pred_rf, average=None, labels=range(len(train['ID'].unique())))
recall = recall_score(y_test, y_pred_rf, average=None, labels=range(len(train['ID'].unique())))
f1 = f1_score(y_test, y_pred_rf, average=None, labels=range(len(train['ID'].unique())))

# print('accuracy:',accuracy)
# print('precision:',list(precision))
# print('recall:',list(recall))
# print('f1_score:',list(f1))

accuracy_dict['RF'] = accuracy
accuracy_dict['RF'].append(accuracy_rf)

precision_dict['RF'] = list(precision)
precision_dict['RF'].append(precision_rf)

recall_dict['RF'] = list(recall)
recall_dict['RF'].append(recall_rf)

f1_score_dict['RF'] = list(f1)
f1_score_dict['RF'].append(f1_rf)

In [ ]:
xgb = XGBClassifier(booster='gbtree', n_estimators=500)
xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb, average='macro')
recall_xgb = recall_score(y_test, y_pred_xgb, average='macro')
f1_xgb = f1_score(y_test, y_pred_xgb, average='macro')

print('accuracy_xgb = ',accuracy_xgb)
print('precision_xgb = ',precision_xgb)
print('recall_xgb = ',recall_xgb)
print('f1_xgb = ',f1_xgb)
print()

accuracy = get_accuracy(y_test, y_pred_xgb)
precision = precision_score(y_test, y_pred_xgb, average=None, labels=range(len(train['ID'].unique())))
recall = recall_score(y_test, y_pred_xgb, average=None, labels=range(len(train['ID'].unique())))
f1 = f1_score(y_test, y_pred_xgb, average=None, labels=range(len(train['ID'].unique())))

# print('accuracy:',accuracy)
# print('precision:',list(precision))
# print('recall:',list(recall))
# print('f1_score:',list(f1))

accuracy_dict['xgb'] = accuracy
accuracy_dict['xgb'].append(accuracy_xgb)

precision_dict['xgb'] = list(precision)
precision_dict['xgb'].append(precision_xgb)

recall_dict['xgb'] = list(recall)
recall_dict['xgb'].append(recall_xgb)

f1_score_dict['xgb'] = list(f1)
f1_score_dict['xgb'].append(f1_xgb)

In [ ]:
y_pred_list = [y_pred_SVM, y_pred_rf, y_pred_xgb]
confusion_matrix_name = ['SVM Confusion Matrix', 'RandomForest Confusion Matrix', 'XGBoost Confusion Matrix']

for i in range(len(y_pred_list)):
    confusion_matrix_temp = confusion_matrix(y_test, y_pred_list[i])
    cm_percentage = confusion_matrix_temp.astype('float') / confusion_matrix_temp.sum(axis=1)[:, np.newaxis]
    
    plt.figure(figsize=(7, 5.5))
    sns.heatmap(cm_percentage, annot=True, fmt='.2f', cmap='Blues', xticklabels=sheet_name, yticklabels=sheet_name)

    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(confusion_matrix_name[i])
#     plt.savefig(f'22elements_{confusion_matrix_name[i]}.png', dpi=500, bbox_inches='tight')
    plt.savefig(f'35elements_{confusion_matrix_name[i]}.png', dpi=500, bbox_inches='tight')    

In [ ]:
# joblib.dump(SVM, '22elements_svm.pkl')
# joblib.dump(rf, '22elements_rf.pkl')
# joblib.dump(xgb, '22elements_xgb.pkl')

joblib.dump(SVM, '35elements_svm.pkl')
joblib.dump(rf, '35elements_rf.pkl')
joblib.dump(xgb, '35elements_xgb.pkl')

In [ ]:
import shap

explainer = shap.TreeExplainer(xgb, X_train)
shap_values = explainer.shap_values(X_test)

# elements_name = ['TiO2','P2O5','Nb','Ta','Zr','Hf','Y','La','Ce','Pr','Nd','Sm','Eu','Gd','Ho','Er','Yb','Lu','Dy','Tb','Cr','Ni']
elements_name = ['SiO2','TiO2','Al2O3','FeOt','CaO','MgO','MnO','K2O','Na2O','P2O5','Rb','Sr','Ba','Th','U','Nb','Ta','Zr','Hf','Y','La','Ce','Pr','Nd','Sm','Eu','Gd','Ho','Er','Yb','Lu','Dy','Tb','Cr','Ni']

In [ ]:
# max_display = 22
max_display = 35

for class_index in range(len(sheet_name)):
    shap.plots.violin(shap_values[class_index], features=X_test, feature_names=elements_name, max_display=max_display, plot_type="violin", show=False)
    plt.title(f'SHAP Values for {sheet_name[class_index]}')
    # plt.savefig(f'22elements_XGBoost SHAP Values for {sheet_name[class_index]}.png', dpi=500, bbox_inches='tight')
    plt.savefig(f'35elements_XGBoost SHAP Values for {sheet_name[class_index]}.png', dpi=500, bbox_inches='tight')
    plt.show()

In [ ]:
# 可视化所有类别的 SHAP 汇总图
shap.summary_plot(shap_values, X_test, feature_names=elements_name, max_display=max_display, class_names=sheet_name, plot_type='bar', show=False)
plt.title('XGBoost SHAP Summary')
# plt.savefig('22elements_XGBoost_shap_summary_plot.png', dpi=500, bbox_inches='tight')
plt.savefig('35elements_XGBoost_shap_summary_plot.png', dpi=500, bbox_inches='tight')
plt.show()